In [2]:
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence



import sys
if ".." not in sys.path:
    sys.path.insert(0, "..")

from datasets import create_batchdict, PlayByPlayDataset

dataset = PlayByPlayDataset("../data/nfl-big-data-bowl-2024/play_by_play_val.pkl")


torch.__version__


'2.1.0'

In [8]:
import numpy as np

y = np.array([dataset[i]['yards_after_contact'] for  i  in range(len(dataset))])
np.mean(y)


3.4594572025052197

In [8]:
dataset[0]["play_features_encoded"]


Q1                       0.000000
Q2                       0.000000
Q3                       0.000000
Q4                       1.000000
OT                       0.000000
down_1                   0.000000
down_2                   0.000000
down_3                   1.000000
down_4                   0.000000
yardage_short            0.000000
yardage_medium           1.000000
yardage_long             0.000000
yardage_very_long        0.000000
time                     3.066667
preSnapHomeScore        19.000000
preSnapVisitorScore      3.000000
0                       16.000000
formation_EMPTY          0.000000
formation_I_FORM         0.000000
formation_JUMBO          0.000000
formation_PISTOL         0.000000
formation_SHOTGUN        0.000000
formation_SINGLEBACK     1.000000
formation_WILDCAT        0.000000
n_def_in_box_(0, 4]      0.000000
n_def_in_box_(4, 5]      0.000000
n_def_in_box_(5, 6]      0.000000
n_def_in_box_(6, 7]      1.000000
n_def_in_box_(7, 8]      0.000000
n_def_in_box_(

In [10]:
def collate_padded_play_data_with_context(batch):
    batchdict = create_batchdict(batch)
    X_geometric = torch.cat([pad_sequence(batchdict[k], batch_first=True, padding_value=PAD_VALUE) for k in GEOMETRIC_KEYS + RAW_KEYS], dim=2)
    X_ball_carrier = pad_sequence(batchdict["ball_carrier_raw"], batch_first=True, padding_value=PAD_VALUE)
    X_tacklers, n_tacklers = pad_tacklers(batchdict["tacklers_raw"])
    import pdb; pdb.set_trace()
    X_padded_static = None # TODO: tile in time, then pad_sequence
    return {
        "time_series_features": (X_geometric, X_ball_carrier, X_tacklers, n_tacklers),
        "features": X_padded_static,
        "target": torch.tensor(batchdict[TARGET_KEY], dtype=torch.float),
        "treatment": torch.tensor(batchdict[TREATMENT_KEY], dtype=torch.float),
    }

dataloader = DataLoader(dataset, batch_size=4, shuffle=False, collate_fn=collate_padded_play_data_with_context)
batch = next(iter(dataloader))


> c:\users\trent\appdata\local\temp\ipykernel_2044\2877267107.py(7)collate_padded_play_data_with_context()

[]
dict_keys(['offense_geometric', 'defense_geometric', 'offense_raw', 'defense_raw', 'ball_carrier_raw', 'tacklers_raw', 'event_timeseries', 'tackle_successful', 'yards_after_contact', 'play_features_encoded'])


In [14]:
batch["time_series_features"][3].size()


torch.Size([4])

In [4]:
test_batch_size = 4
sample_batchdict = create_batchdict([dataset[i] for i in range(test_batch_size)])

[t.view(t.size(0), -1, 6).size() for t in sample_batchdict["tacklers_raw"]]


[torch.Size([43, 1, 6]),
 torch.Size([21, 2, 6]),
 torch.Size([16, 1, 6]),
 torch.Size([47, 1, 6])]

In [16]:
(batch['time_series_features'][0].size(),
batch['time_series_features'][1].size(),
batch['time_series_features'][2].size(),
batch['time_series_features'][3],
)


(torch.Size([4, 47, 105]),
 torch.Size([4, 47, 6]),
 torch.Size([4, 47, 2, 6]),
 tensor([1, 2, 1, 1]))

In [59]:
# 248 is the dimensionality of the time-series features

encoder_layer = nn.TransformerEncoderLayer(d_model=248, nhead=8, batch_first=True)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)
X = batch["time_series_features"].float()
out = transformer_encoder(X, src_key_padding_mask=(X[..., 0] == PAD_VALUE))


In [60]:
out.size() # batch size, time, n_features
# in the future -- concat with batch_size, n_features' shaped things -- need to squish out the time dim


torch.Size([8, 39, 248])

In [61]:
39 * 248


9672